# Customizing the Legend

In [1]:
import pandas as pd

from lets_plot import *

In [2]:
LetsPlot.setup_html()

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/mpg.csv")
print(df.shape)
df.head()

(234, 12)


,Unnamed: 0,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
0,1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
1,2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
2,3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
3,4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
4,5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


## Scale Parameters Related to the Legend

In [4]:
# Default legend
p1 = ggplot(df, aes("cty", "hwy")) + geom_point(aes(color="cyl"))
p1

In [5]:
# Legend name
p1 + scale_color_continuous(name="Number of cylinders")

In [6]:
# Breaks and labels
p1 + scale_color_continuous(breaks=[4, 5, 6, 8], labels=["four", "five", "six", "eight"])

In [7]:
# Format labels
p1 + scale_color_continuous(format="{} cylinders\nin the vehicle’s engine")

## Guides

### `guide_legend()`

In [8]:
# Default categorical legend
p2 = ggplot(df, aes("displ", "hwy", color="manufacturer")) + geom_point(size=5)
p2

In [9]:
# Legend name
p2 + scale_color_discrete(guide=guide_legend("Company"))

In [10]:
# Layout the legend in two columns
p2 + scale_color_discrete(guide=guide_legend(ncol=2))

In [11]:
# Fill by rows
p2 + scale_color_discrete(guide=guide_legend(ncol=2, byrow=True))

### `guide_colorbar()`

In [12]:
# Default color legend
p3 = ggplot(df, aes("displ", "hwy")) + geom_point(aes(color="cty"))
p3

In [13]:
# Legend name
p3 + scale_color_continuous(guide=guide_colorbar("City mileage"))

In [14]:
# Adjust colorbar size
p3 + scale_color_continuous(guide=guide_colorbar(barwidth=10, barheight=200))

In [15]:
# Fewer bins
p3 + scale_color_continuous(breaks=[13, 22, 31], guide=guide_colorbar(nbin=3))

### `guides()`

In [16]:
# Default complex legend
p4 = ggplot(df, aes("displ", "hwy")) + geom_point(aes(color="cty", shape="drv"), size=5)
p4

In [17]:
# Guides for 'color' and 'shape' aesthetics
p4 + guides(color=guide_colorbar(barwidth=10), shape=guide_legend(ncol=2))

### Customizing Legend Appearance with `override_aes`

In [18]:
p4 + guides(shape=guide_legend(override_aes={'size': 8, 'color': "#f03b20"}))

## Legend Position

In [19]:
# Draw legend below the plot area
p4 + theme(legend_position='bottom')

In [20]:
# Specify position, justification and orientation
p4 + theme(legend_position=[1, 1], legend_justification=[1, 1], legend_direction='horizontal')

## Custom Legend with `manual_key` Option

The `manual_key` option allows you to create custom legend entries specific to individual plot layers.

The `manual_key` option can be used in two ways:

- Simple case: pass a string to serve as the label for the legend entry.

- Advanced case: use the `layer_key()` function for more detailed customization:

  - `label` - text for the legend element
  - `group` - key used to group elements in the legend
  - `index` - position of the element within its legend group
  - `kwargs` - dictionary of aesthetic parameters to be applied in the legend

In [21]:
# Default plot
ggplot(df, aes("displ", "hwy")) + \
    geom_point(color="#878787") + \
    geom_smooth(method='loess', se=False, color="#2166ac") + \
    geom_smooth(method='lm', se=False, color="#b2182b")

### Simple Custom Legend

In [22]:
ggplot(df, aes("displ", "hwy")) + \
    geom_point(color="#878787",
               manual_key="Observed Data"                   # label for the legend entry
              ) + \
    geom_smooth(method='loess', se=False, color="#2166ac",
                manual_key="LOESS Trend Line") + \
    geom_smooth(method='lm', se=False, color="#b2182b",
                manual_key="Linear Model Trend Line") + \
    ggsize(800, 400)

### The `layer_key()` Function

All constants for the visual representation of the legend key are inherited from the geometry.
Use the `layer_key()` function to override these values.
You can also specify the legend group in which the element is displayed and the position of the element within the legend group.

In [23]:
p5 = ggplot(df, aes("displ", "hwy")) + \
    geom_point(color="#878787",
               manual_key=layer_key("Observed Data",          # <-- label for the legend entry
                                    color='pen', size=8)      # <-- override aesthetics
              ) + \
    geom_smooth(method='loess', se=False, color="#2166ac",
                manual_key=layer_key("LOESS", 
                                     group="Methods",         # <-- assign a group (note the same group in the layer below)
                                     index=1,                 # <-- position of the element
                                     size=4)) + \
    geom_smooth(method='lm', se=False, color="#b2182b",
                manual_key=layer_key("LM", group="Methods", index=0, size=4)) + \
    ggsize(800, 400)
p5

### Change Title of Custom Legends

In [24]:
p5 + labs(
    manual="Scatter Plot",  # <-- "manual" is a default 'key' referring to a custom legend
    Methods="Trend Lines"   # <-- "Methods" is the group name referring to the second custom legend
)

## Merge Legend Entries in Custom Legend

In [25]:
ggplot(mapping=aes(as_discrete('class', order_by='..y..'), 'hwy')) + \
    stat_summary(data=df[df.year == 1999], fun='mean', # <-- 1999
                 geom='line', size=2, color='#D87093',
                 manual_key="1999"                     # <-- label for the legend entry
                ) + \
    stat_summary(data=df[df.year == 1999], fun='mean',
                 geom='point', size=5, shape=22,
                 fill='#D87093', color='paper',
                 manual_key=layer_key("1999",          # <-- Using the same label as above results in a composite entry in the legend
                                      size=3)) + \
    stat_summary(data=df[df.year == 2008], fun='mean', # <-- 2008
                 geom='line', size=2, color='#708090',
                 manual_key="2008") + \
    stat_summary(data=df[df.year == 2008], fun='mean',
                 geom='point', size=7, shape=23, fill='#708090', color='paper',
                 manual_key=layer_key("2008", size=5)) + \
    guides(manual=guide_legend(
               override_aes=dict(stroke=0,             # <-- remove `stroke` in the "manual" legend
                                 size=1)               # <-- update `size` if not specified otherwise in the layer itself
          )) + \
    ggsize(800, 400) + \
    theme(legend_position=[0.9, 0.95], legend_justification=[1, 1],
          legend_direction='horizontal',
          legend_background=element_rect(size=1, linetype='dotted'))